In [1]:
import json
import pickle
from pathlib import Path
from typing import List

import cv2 as cv
import numpy as np
from sklearn.mixture import BayesianGaussianMixture

In [2]:
laser_samples_path = Path('spots')
pkl_model_path = Path('spot_model.pkl')
json_model_path = Path('spot_model.json')

In [3]:
laser_samples = laser_samples_path.rglob('*.jpg', case_sensitive=False)

In [4]:
imgs = [cv.imread(path) for path in laser_samples]

In [5]:
hsv_images: List[cv.Mat] = [cv.cvtColor(img, cv.COLOR_BGR2HSV) for img in imgs]

In [6]:
flattened_image = np.empty((0, 3), dtype=np.float32)
for img in hsv_images:
    flattened_image = np.vstack((flattened_image, img.reshape(-1, 3).astype(np.float32)))

In [7]:
gmm = BayesianGaussianMixture(
    n_components=10,
    max_iter=200,
    covariance_type='full'
)
gmm.fit(flattened_image)

BayesianGaussianMixture(max_iter=200, n_components=10)

In [8]:
with open(pkl_model_path, 'wb') as handle:
    pickle.dump(gmm, handle)

In [9]:
len(gmm.means_)

10

In [10]:
for cov in gmm.covariances_:
    print(cov)

[[ 14.61296875  -6.27724385 -20.63476337]
 [ -6.27724385 913.06235014 674.93463499]
 [-20.63476337 674.93463499 690.88970785]]
[[ 69.89629849  62.55235709  27.5749425 ]
 [ 62.55235709 518.85959355 255.42879913]
 [ 27.5749425  255.42879913 179.9920904 ]]
[[118.16769114 -26.26223797 -60.05801112]
 [-26.26223797 243.74678471 230.14219062]
 [-60.05801112 230.14219062 595.73679646]]
[[1606.18453479 -317.49122587 -202.52495464]
 [-317.49122587  592.1011429   574.32459253]
 [-202.52495464  574.32459253  819.31205941]]
[[ 25.91013907 -20.55589285  30.17736617]
 [-20.55589285 102.09380688  45.2479121 ]
 [ 30.17736617  45.2479121  559.85959104]]
[[135.13515016  18.39928016  41.68373791]
 [ 18.39928016 461.36056901 363.38903908]
 [ 41.68373791 363.38903908 511.03959812]]
[[1606.18449535 -317.4912866  -202.52494049]
 [-317.4912866   592.10091294  574.32440377]
 [-202.52494049  574.32440377  819.31185775]]
[[ 26.24539988  -6.43136448  -9.28238814]
 [ -6.43136448 456.21046022 138.30626473]
 [ -9.282

In [11]:
with open(json_model_path, 'w', encoding='utf-8') as handle:
    json.dump({
        'weights': gmm.weights_.tolist(),
        'means': gmm.means_.tolist(),
        'covariances': gmm.covariances_.tolist()
    }, handle)